<a href="https://colab.research.google.com/github/crisubianca/RN-2022/blob/main/Identificare_Fantome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Loading libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")

# Multilayer perceptron Neural Network
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

In [ ]:
# importing data
traindata = pd.read_csv('../content/train.csv')
testdata = pd.read_csv('../content/test.csv')
print(traindata.head())
print(testdata.head())

   id  bone_length  rotting_flesh  hair_length  has_soul  color    type
0   0     0.354512       0.350839     0.465761  0.781142  clear   Ghoul
1   1     0.575560       0.425868     0.531401  0.439899  green  Goblin
2   2     0.467875       0.354330     0.811616  0.791225  black   Ghoul
3   4     0.776652       0.508723     0.636766  0.884464  black   Ghoul
4   5     0.566117       0.875862     0.418594  0.636438  green   Ghost
   id  bone_length  rotting_flesh  hair_length  has_soul  color
0   3     0.471774       0.387937     0.706087  0.698537  black
1   6     0.427332       0.645024     0.565558  0.451462  white
2   9     0.549602       0.491931     0.660387  0.449809  black
3  10     0.638095       0.682867     0.471409  0.356924  white
4  13     0.361762       0.583997     0.377256  0.276364  black


In [ ]:
# encoding string data columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
traindata['color'] = le.fit_transform(traindata['color'])
testdata['color'] = le.fit_transform(testdata['color'])
traindata['type'] = le.fit_transform(traindata['type'].astype(str))


0      1
1      2
2      1
3      1
4      0
      ..
366    2
367    0
368    1
369    0
370    1
Name: type, Length: 371, dtype: int64


In [ ]:
# Getting required features
X = traindata.iloc[:,1:6].values
y = traindata.iloc[:,6].values
X_test = testdata.iloc[:,1:6].values
print(X.shape)
print(X_test.shape)

(371, 5)
(529, 5)


In [ ]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X)
X_test = sc.fit_transform(X_test)

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# Using one hot encoding for multi-class categories (Fast-ANN)
y = np_utils.to_categorical(y)

In [ ]:
# Defining keras model
# create model
model = Sequential()
model.add(Dense(5, input_dim=5, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# build the model
# Fit the model
model.fit(X_train, y, epochs=200, batch_size=5, verbose=0)

In [ ]:
# predicting results
pred = model.predict(X_test)
pred = pred.argmax(1)
pred = le.inverse_transform(pred)

17/17 [==============================] - 0s 2ms/step


In [ ]:
# generating output
id_data = testdata[['id']].values
type_data = np.expand_dims(pred,axis=1)
f = np.hstack((id_data,type_data))
df = pd.DataFrame(f, columns = ['id', 'type'])
df.to_csv('../content/sample_submission.csv', index=False)
df.head()

,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Ghoul
4,13,Ghost
